In [ ]:
"""
 author: Prashant Prasad Kanth
 date: 20/09/2022 'MM/DD/YYY'
"""

In [ ]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
import re

In [ ]:
spark = SparkSession.builder.appName("SparkStreaming").getOrCreate()
sc = spark.sparkContext
ssc = StreamingContext(sc, 1)

In [ ]:
def parse(line):
    match=re.search("Dm=(\d+)",line) # Search for required match condition 
    if match: # if there is a match
        val=match.group(1)
        return [int(val)] # return that value as a list of one element 
    return [] # else return none

In [ ]:
def display(rdd, name, sep):
        print(f"{name}: {rdd.collect()[0]}", end=sep)

In [ ]:
def stats():
    # perform required transformations to calculate stats of a window
    minValue = windowData.reduce(lambda a, b: min(a, b))
    maxValue = windowData.reduce(lambda a, b: max(a, b))
    average = windowData.map(lambda a: (a, 1))\
                        .reduce(lambda a, b: (a[0] + b[0] , a[1]+b[1]))\
                        .map(lambda a: a[0]/a[1])
    # print the stats
    minValue.foreachRDD(lambda rdd: display(rdd, 'min', sep=", "))
    maxValue.foreachRDD(lambda rdd: display(rdd, 'max', sep=", "))
    average.foreachRDD(lambda rdd: display(rdd, 'avg', sep="\n"))

In [ ]:
lines = ssc.socketTextStream("ip","portnumber") # replace ip and portnumber with appropriate values
vals = lines.flatMap(parse)
# get a window of size 10 and sliding interval 5
windowData = vals.window(10, 5)
# print contents in a window
windowData.pprint()
stats()

In [ ]:
ssc.start()
ssc.awaitTermination()

In [ ]:
ssc.stop()